<a href="https://colab.research.google.com/github/sharangw/FakeNewsGenerator/blob/master/FakeHeadlineGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

tf.enable_eager_execution()

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Data Cleaning

In [0]:
# read the data file

df=pd.read_csv("drive/My Drive/Colab Notebooks/abcnews-date-text.csv")

In [7]:
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [0]:
## create a list containing the text of the articles

y=[]
for i in df.loc[:,"headline_text"].values:
    y.append(i)

In [9]:
y[0:4]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise']

In [11]:
len(y)

1186018

In [0]:
concatstr = ' '.join(y) ## take the contents of the list and put it into one giant string

In [13]:
len(concatstr)

49546647

Can Change This Number of Characters:

In [0]:
newinput = concatstr[0:15000000] # work with the first 30 million characters
text = newinput

In [15]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

39 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [20]:
print('{')
for char,_ in zip(char2idx, range(120)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '.' :   1,
  '0' :   2,
  '1' :   3,
  '2' :   4,
  '3' :   5,
  '4' :   6,
  '5' :   7,
  '6' :   8,
  '7' :   9,
  '8' :  10,
  '9' :  11,
  ';' :  12,
  'a' :  13,
  'b' :  14,
  'c' :  15,
  'd' :  16,
  'e' :  17,
  'f' :  18,
  'g' :  19,
  'h' :  20,
  'i' :  21,
  'j' :  22,
  'k' :  23,
  'l' :  24,
  'm' :  25,
  'n' :  26,
  'o' :  27,
  'p' :  28,
  'q' :  29,
  'r' :  30,
  's' :  31,
  't' :  32,
  'u' :  33,
  'v' :  34,
  'w' :  35,
  'x' :  36,
  'y' :  37,
  'z' :  38,
  ...
}


In [21]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:15]), text_as_int[:15]))

'aba decides aga' ---- characters mapped to int ---- > [13 14 13  0 16 17 15 21 16 17 31  0 13 19 13]


## Model Building

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [23]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'aba decides against community broadcasting licence act fire witnesses must be aware of defamation a g calls for infrastructure protection summit air nz staff in aust strike for pay rise air nz strike t'
'o affect australian travellers ambitious olsson wins triple jump antic delighted with record breaking barca aussie qualifier stosur wastes four memphis match aust addresses un security council over ira'
'q australia is locked into war timetable opp australia to contribute 10 million in aid to iraq barca take record as robson celebrates birthday in bathhouse plans move ahead big hopes for launceston cyc'
'ling championship big plan to boost paroo water supplies blizzard buries united states in bills brigadier dismisses reports troops harassed in british combat troops arriving daily in kuwait bryant lead'
's lakers to double overtime win bushfire victims urged to see centrelink businesses should prepare for terrorist attacks calleri avenges final defeat to eliminate massu call for ethan

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [25]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((128, 200), (128, 200)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 200, 39) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (128, None, 256)          9984      
_________________________________________________________________
gru_2 (GRU)                  (128, None, 1024)         3935232   
_________________________________________________________________
gru_3 (GRU)                  (128, None, 1024)         6294528   
_________________________________________________________________
dense_2 (Dense)              (128, None, 39)           39975     
Total params: 10,279,719
Trainable params: 10,279,719
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([38, 31, 34, 19, 37,  6, 33, 23, 14, 20,  1, 28, 20,  3, 37, 22, 26,
       15,  0,  3, 20, 36, 22, 17, 13,  7, 30, 31, 24, 23, 33, 12, 34, 18,
       26, 34, 15, 27, 14,  5,  3, 19,  9, 31, 37,  6, 14, 10, 14,  1, 32,
       10, 25,  6,  3, 10, 38, 32, 23, 13, 17, 19, 22, 17, 37, 22, 38, 22,
        1, 20, 18,  9, 11,  2, 26, 25,  5, 16, 37, 14, 12, 34, 36,  6, 35,
       26, 22, 23, 24, 36,  9,  0, 10, 22, 17,  8,  6, 13, 31, 17, 32, 10,
       18, 31, 11, 21,  5, 12, 23, 20, 38,  3, 29, 23, 10, 28, 20, 21,  8,
        5, 10, 25,  7, 29, 38, 14,  6,  9, 18,  6, 17, 22,  2,  7, 10, 11,
        2, 31, 23, 33, 11, 20,  5, 18, 38,  9, 25, 27, 17,  4, 10, 35, 15,
       11, 18, 17, 36,  5, 26, 19, 38,  3, 12, 37, 13, 15, 20,  1, 14, 14,
        0, 26, 20, 36, 33, 25, 23, 19, 18, 17, 23, 29, 16, 34,  4, 10, 11,
       12, 22, 14, 14, 30, 34, 33, 20,  0, 31, 35, 33, 16])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'decision to chop trees council feels fenced in over qr plan councillors urged to consider roles during court rules against minister in deportation case court to hear high rise concerns crean claims 15'

Next Char Predictions: 
 'zsvgy4ukbh.ph1yjnc 1hxjea5rslku;vfnvcob31g7sy4b8b.t8m418ztkaegjeyjzj.hf790nm3dyb;vx4wnjklx7 8je64aset8fs9i3;khz1qk8phi638m5qzb47f4ej05890sku9h3fz7moe28wc9fex3ngz1;yach.bb nhxumkgfekqdv289;jbbrvuh swud'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Train Model

In [0]:
EPOCHS=15

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
583/583 [==============================] - 661s 1s/step - loss: 1.8660
Epoch 2/15
583/583 [==============================] - 656s 1s/step - loss: 1.2907
Epoch 3/15
583/583 [==============================] - 652s 1s/step - loss: 1.2075
Epoch 4/15
583/583 [==============================] - 652s 1s/step - loss: 1.1567
Epoch 5/15
583/583 [==============================] - 653s 1s/step - loss: 1.1189
Epoch 6/15
583/583 [==============================] - 657s 1s/step - loss: 1.0895
Epoch 7/15
583/583 [==============================] - 656s 1s/step - loss: 1.0670
Epoch 8/15
583/583 [==============================] - 656s 1s/step - loss: 1.0505
Epoch 9/15
583/583 [==============================] - 656s 1s/step - loss: 1.0378
Epoch 10/15
583/583 [==============================] - 657s 1s/step - loss: 1.0274
Epoch 11/15
583/583 [==============================] - 658s 1s/step - loss: 1.0187
Epoch 12/15
583/583 [==============================] - 659s 1s/step - loss: 1.0115
Epoch 13/15
5

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_5'

In [0]:
from google.colab import files
files.download('./training_checkpoints/ckpt_10.index') 
files.download('./training_checkpoints/ckpt_10.data-00000-of-00002') 
files.download('./training_checkpoints/ckpt_10.data-00001-of-00002')

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # './training_checkpoints/ckpt_30'
model.load_weights('ckpt_15')

model.build(tf.TensorShape([1, None]))

## Text Generating

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 50

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in tqdm(range(num_generate)):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            9984      
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           6294528   
_________________________________________________________________
dense_4 (Dense)              (1, None, 39)             39975     
Total params: 10,279,719
Trainable params: 10,279,719
Non-trainable params: 0
_________________________________________________________________


In [37]:
print(generate_text(model, start_string=u"experts claim "))

print(generate_text(model, start_string=u"yesterday in russia "))

print(generate_text(model, start_string=u"the news coming from "))


  2%|▏         | 1/50 [00:00<00:08,  5.99it/s]

experts claim 120000 role golden travel chapman jets pm rewarded


  2%|▏         | 1/50 [00:00<00:08,  6.11it/s]

yesterday in russia budget may spark biofuel plant move nt govt ridist


100%|██████████| 50/50 [00:01<00:00, 42.79it/s]

the news coming from the charge lost voters on knights lead murray magi


## Fake Headlines:

experts claim 6000 fine quest over mass keelty plans pizza emerg

yesterday in russia ballarat publisher on display in pakistan 15e pres

the news coming from the head india to limit dire from polls on track f

breaking news labor leader tightens overhaul eagles new interce

obama set to be more ceremony recommended for remote sch

schools request conviction since jailed final tensions imf

government inquiry told aust maoiks for 10 year hollow highli

president inspects japanese gymnastics chance stallone plead

people to challenge bush secretary leader johnson new bhe

celebrity leaders agree to meeting harbour bridge federal go

united states to focus on climate uniform first cut bligh rack c




In [63]:
print(generate_text(model, start_string=u"united states "))

100%|██████████| 50/50 [00:01<00:00, 44.30it/s]

united states to focus on climate uniform first cut bligh rack c
